In [80]:
import pandas as pd
import numpy as np

In [81]:
df = pd.DataFrame({"A": [1,2,3,4,5,6,7], "B": [1,2,np.nan,4,np.nan,np.nan,np.nan]})
df2 = pd.DataFrame({"A": [1,2,3,4,5,6,7], "B": [1,2,3,4,5,6,np.nan]})
df3 = pd.DataFrame({"A": [1,2,3,4,5,6,7], "B": [1,2,3,4,5,6,7]})

In [82]:
df.head()

,A,B
0,1,1.0
1,2,2.0
2,3,NaN
3,4,4.0
4,5,NaN


In [83]:
df2.head()

,A,B
0,1,1.0
1,2,2.0
2,3,3.0
3,4,4.0
4,5,5.0


In [84]:
def non_or_empty(x):
    def bitwise_non_or_empty(x):
        boo = False
        if np.isnan(x): 
            boo = True
        if x == None:
            boo = True
        if x == '':
            boo = True
        return boo
    boo = x.apply(
        lambda x: bitwise_non_or_empty(x))
    return boo

In [91]:
def join_and_update(left, right, left_on, right_on,
                    left_update, right_update, joined_on=""):
    df_join = left.merge(right, how="left",
                         left_on=left_on, right_on=right_on,
                         suffixes=['','___y'])
    check = (non_or_empty(
        df_join[left_update])) & (~non_or_empty(
        df_join[right_update+'___y']))
    if 'Joined_on' not in df_join.columns:
        df_join['Joined_on'] = np.nan
    df_join.loc[check, 'Joined_on'] = joined_on
    df_join.loc[check, left_update] = df_join[right_update + '___y']
    df_join = df_join[[i for i in df_join.columns if not i.endswith('___y')]]
    return df_join

In [92]:
df_join = join_and_update(df, df2, left_on="A",
                          right_on="A", left_update="B",
                          right_update="B", joined_on="Joined")

In [93]:
join_and_update(df_join, df3, left_on="A",
                right_on="A", left_update="B",
                right_update="B", joined_on="Joinered")

,A,B,Joined_on
0,1,1.0,NaN
1,2,2.0,NaN
2,3,3.0,Joined
3,4,4.0,NaN
4,5,5.0,Joined
5,6,6.0,Joined
6,7,7.0,Joinered


In [98]:
(df.pipe(join_and_update,
        right = df2,
        left_on="A", right_on="A",
        left_update="B", right_update="B",
        joined_on="Joined")
     .pipe(join_and_update,
        right = df3,
        left_on="A", right_on="A",
        left_update="B", right_update="B",
        joined_on="Joined"))

,A,B,Joined_on
0,1,1.0,NaN
1,2,2.0,NaN
2,3,3.0,Joined
3,4,4.0,NaN
4,5,5.0,Joined
5,6,6.0,Joined
6,7,7.0,Joined
